<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Summarize_KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gluonnlp mxnet tqdm
!pip install sentencepiece==0.1.85
!pip install torch==1.6.0
!pip install transformers==2.1.1
!pip install kss

     |████████████████████████████████| 348kB 4.3MB/s 
     |████████████████████████████████| 55.0MB 53kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588529 sha256=9a61483cfc3e7653ab294a78df9474d98de4ca610524ed31bdf9de29e1d55fae
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: You must give at least one requirement to install (see "pip help install")
     |████████████████████████████████| 1.0MB 5.7MB/s 
     |████████████████████████████████| 317kB 4.6MB/s 
     |████████████████████████████████| 133kB 10.3MB/s 
     |████████████████████████████████| 890kB 14.6MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 6.7MB 26.6MB/s 
  Created wheel for sacremoses: filename=sacr

In [4]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git

  Cloning https://github.com/SKT-AI/KoGPT2.git to /tmp/pip-req-build-406rcmqj
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-406rcmqj
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=47a6375a149ac08374b55f8468fbc8bf022f62c1e20f06db72c482d0e108dfa2
  Stored in directory: /tmp/pip-ephem-wheel-cache-60c5z_s3/wheels/3b/a2/30/432bb7490a2ea23a90049e6c5725f6acd7e925f1abfb3d7ddf
Successfully built kogpt2


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from gluonnlp.data import SentencepieceTokenizer 
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

In [13]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kogpt2.utils import download, tokenizer, get_tokenizer
from kogpt2.pytorch_kogpt2 import GPT2Config, GPT2LMHeadModel

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [28]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch = 50  # 학습 epoch
save_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

In [9]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.load_state_dict(torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')

In [14]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

using cached model


In [15]:
data_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/DataSet/Topic_keywords.csv'

In [16]:
news_data = pd.read_csv(data_path, encoding='utf-8')
news_data.head()

,Date,Topic_keywords,Text
0,2006-06-27,"['삼성전자', '하이닉스', '메모리', '긍정', '증권', '업체', '분기'...","미래에셋증권은 삼성전자[005930]는 2.4분기, 하이닉스[000660]는 1...."
1,2006-06-27,"['하이닉스', '발행', '주가', '신주', '매각', '외국', '이번', '...",하이닉스반도체가 블록딜을 마치자마자 주가가 급등했다. 하이닉스 주가는 26일 유가...
2,2006-06-27,"['외국인', '매도', '상승', '종목', '이날', '순매도', '하락', '...",코스피지수가 엿새 만에 1240선을 회복했다. 27일 코스피지수는 전날 미국 증시...
3,2006-06-27,"['지수', '이날', '상승', '종목', '시장', '프로그램', '순매도', ...",코스피지수가 하락 하루 만에 반등하며 1240선에 다가섰다. 26일 코스피지수는 ...
4,2006-06-27,"['창업', '시장', '원자', '벤처', '미국', '대표', '현미경', '장...",세계시장에 나갔을 때 '코리아 디스카운트'가 이 정도로 심할 줄은 몰랐습니다. 그러...


In [17]:
# 학습, 테스트 데이터 분리
train_data = news_data.sample(frac=0.8, random_state=2020)
test_data = news_data.drop(train_data.index)
len(train_data), len(test_data)

(80, 20)

In [18]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(80, 20)

In [19]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['Topic_keywords'][i], train_data['Text'][i]]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['Topic_keywords'][i], test_data['Text'][i]])
if len()>=10:
    [:10]
dataset_test[:5], dataset_train[:5]

100%|██████████| 20/20 [00:00<00:00, 8298.16it/s]


([["['지수', '이날', '상승', '종목', '시장', '프로그램', '순매도', '매수세', '기록', '금융']",
   "코스피지수가 하락 하루 만에 반등하며 1240선에 다가섰다.  26일 코스피지수는 전거래일보다 9.43포인트 상승한 1238.05에 장을 마감했다.  이번주 예정된 미국 연방공개시장위원회 회의를 앞두고 투자자들의 관망세가 이어지며 지수가 보합권에서 혼조세를 나타냈다. 외국인 순매도세가 14일째 이어진 탓에 장중 1221선까지 밀려나기도 했지만 프로그램 매수세가 1000억원 이상 유입된 데 힘입어 상승 반전하며 1230선을 다시 회복했다.  투자 주체별로 보면 이날 외국인은 1832억원 순매도세를 기록하며 14일째 연속 '팔자'를 나타냈다. 개인도 149억원 순매도세를 기록했다. 이에 비해 기관은 757억원 순매수세를 나타냈다. 또 이날 프로그램은 1006억원 매수 우위였다.  이날 상승 종목 수는 상한가 8종목을 포함해 361개, 하락 종목 수는 하한가 2종목을 포함해 359개였다.  업종별로 보면 의료정밀, 전기ㆍ전자, 철강 업종이 상승한 반면 유통, 운수창고 등은 비교적 큰 폭으로 하락했다.  무엇보다 이날 시세를 주도한 것은 하반기 실적 낙관론이 퍼진 기술주들이었다.  이날 코스피지수는 뚜렷한 매수 주체가 부재한 가운데 프로그램 매수세가 유입되는 지루한 장세로 출발했으나 삼성전자, 하이닉스와 LG필립스LCD를 비롯한 반도체, 디스플레이 관련 대표주들이 시장 전면에 나서며 지수를 이끌었다.  삼성전자가 58만2000원에 마감, 지난 7일 이후 가장 높은 종가를 기록했다. LG필립스LCD, LG전자, 삼성SDI 등이 모두 상승했으며 특히 채권단 지분 대량 매각이 성사된 하이닉스가 급등하며 흐름을 선도했다.  최근 시장의 주도주 흐름이 통신주에서 반도체, 디스플레이 관련 대표주로 바뀌고 있다는 점은 긍정적인 신호라는 평가도 나오고 있다.  또 포스코, 현대제철, 동국제강 등 철강주들이 모두 강세였다. 그간 중국 차이나유니콤 투자

In [63]:
class GPTDataset(Dataset):
  def __init__(self, data_file, vocab, tokenizer):
    self.data =[]
    self.vocab = vocab
    self.tokenizer = tokenizer

    for i in range(len(data_file)):
      toeknized_line = tokenizer(data_file[i][1][:10])
      index_of_words = [vocab[vocab.bos_token],] + vocab[toeknized_line]+ [vocab[vocab.eos_token]]
      self.data.append(index_of_words)

  def __len__(self):
    return len(self.data)
  def __getitem__(self,index):
    item = self.data[index]

    return item

In [66]:
batch_size = 2
news_dataset = GPTDataset(dataset_train, vocab, sentencepieceTokenizer)
news_data_loader = DataLoader(news_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

In [68]:
for data in news_data_loader:
    print(data)

RuntimeError: ignored

In [60]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [61]:
epoch = 3
for epoch in range(epoch):
  count = 0
  for data in news_data_loader:
    optimizer.zero_grad()
    data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

    data= data.transpose(1,0)
    data= data.to(ctx)
    
    outputs = model(data, labels=data)
    loss, logits = outputs[:2]
    loss.backward()
    optimizer.step()
    if count %10 ==0:
      print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
      # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
      # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
    if (count >0 and count%100==0) or (len(data) < batch_size):
      torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
      }, save_path+'narrativeKoGPT2_checkpoint.tar')

    count += 1

RuntimeError: ignored

In [83]:
load_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/narrativeKoGPT2_checkpoint.tar'
# Device 설정
device = torch.device(ctx)
# 저장한 Checkpoint 불러오기
checkpoint = torch.load(load_path, map_location=device)

# # KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval()

RuntimeError: ignored

In [79]:
toked = sentencepieceTokenizer('지수 이날 상승 종목 시장 프로그램 순매도 매수세 기록 금융')
count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

while 1:
  input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
  predicts = model(input_ids)
  pred = predicts[0]

  last_pred = pred.squeeze()[-1]
  # top_p 샘플링 방법
  # sampling.py를 통해 random, top-k, top-p 선택 가능.
  gen = sampling.top_p(last_pred, vocab, 0.9)
  # gen = sampling.top_k(last_pred, vocab, 5)

  if count>output_size:
    sent += gen.replace('▁', ' ')
    toked = tok(sent)
    count =0
    break
  sent += gen.replace('▁', ' ')
  toked = tok(sent)
  count += 1

for s in kss.split_sentences(sent):
    print(s)

RuntimeError: ignored

In [76]:
toked

['▁지수', '▁이날', '▁상승', '▁종목', '▁시장', '▁프로그램', '▁순매도', '▁매수', '세', '▁기록', '▁금융']